# View all raster data

In [5]:
from collections import defaultdict
import csv
import sys
import time
import gdal
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
import pandas as pd
from descartes.patch import PolygonPatch


import matplotlib.pyplot as plt
%matplotlib inline 


from sklearn.linear_model import SGDClassifier, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import average_precision_score, roc_auc_score
import os
import cv2
from shapely.wkt import loads as wkt_loads
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict

import xgboost as xgb





Using Theano backend.


In [6]:
from __future__ import division

# Overview, 
3 bands     3396*3349*3
Pancromatic  450-800nm     0.31m     pixel 3391*3348
8 multispectral  400-1040nm     1.24m     pixel  848*837*8
8 SWIR 1195-2365nm      7.5m    pixel   136*134*8



In [7]:
gridsize = pd.read_csv('satellite_image/grid_sizes.csv',names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
train_wkt = pd.read_csv('satellite_image/train_wkt_v4.csv')
sample_submission = pd.read_csv('satellite_image/sample_submission.csv')
all_image_ids = gridsize.ImageId.unique()
train_image_ids = train_wkt.ImageId.unique()
test_image_ids = sample_submission.ImageId.unique()

In [9]:
print train_image_ids, type (train_image_ids)

['6040_2_2' '6120_2_2' '6120_2_0' '6090_2_0' '6040_1_3' '6040_1_0'
 '6100_1_3' '6010_4_2' '6110_4_0' '6140_3_1' '6110_1_2' '6100_2_3'
 '6150_2_3' '6160_2_1' '6140_1_2' '6110_3_1' '6010_4_4' '6170_2_4'
 '6170_4_1' '6170_0_4' '6060_2_3' '6070_2_3' '6010_1_2' '6040_4_4'
 '6100_2_2'] <type 'numpy.ndarray'>


In [12]:
train_list = train_image_ids.tolist()

In [54]:
train_list.index('6170_0_4')

19

In [17]:
os.getcwd()

'/Users/tianbaili/Desktop/data science/kaggle'

In [19]:
from gdalconst import *

In [42]:
from gdalconst import *
#current path '/Users/tianbaili/Desktop/data science/kaggle'
#filename = os.path.join( 'satellite_image', 'three_band', '{}.tif'.format(image_id))
#img_filename_3bands = os.getcwd() + '/satellite_image/three_band/6010_0_0.tif'

#filename_3bands = os.getcwd() + '/satellite_image/three_band/6040_2_2.tif'

filename_3bands = os.path.join ('satellite_image', 'sixteen_band', '{}.tif'.format('6040_2_2'+'_M'))
datasetA1 = gdal.Open(filename_3bands, GA_ReadOnly)

print datasetA1.RasterXSize, 'x', datasetA1.RasterYSize, 'x' ,datasetA1.RasterCount

848 x 837 x 8


In [25]:
from gdalconst import *
#current path '/Users/tianbaili/Desktop/data science/kaggle'
#filename = os.path.join( 'satellite_image', 'three_band', '{}.tif'.format(image_id))
#img_filename_3bands = os.getcwd() + '/satellite_image/three_band/6010_0_0.tif'

filename_3bands = os.getcwd() + '/satellite_image/three_band/6040_2_2.tif'

#filename_3bands = os.path.join ('satellite_image', 'three_band', '{}.tif'.format('6040_2_2'))
datasetA1 = gdal.Open(filename_3bands, GA_ReadOnly)

print datasetA1.RasterXSize, 'x', datasetA1.RasterYSize, 'x' ,datasetA1.RasterCount

3391 x 3349 x 3


In [49]:
def view_rastersize (image_id):
    filename_3bands = os.path.join ('satellite_image', 'three_band', '{}.tif'.format(image_id))
    filename_Mbands = os.path.join ('satellite_image', 'sixteen_band', '{}.tif'.format(image_id + '_M'))
    filename_Abands = os.path.join ('satellite_image', 'sixteen_band', '{}.tif'.format(image_id + '_A'))
    filename_Pbands = os.path.join ('satellite_image', 'sixteen_band', '{}.tif'.format(image_id + '_P'))
    data_3bands = gdal.Open(filename_3bands, GA_ReadOnly)
    data_Mbands = gdal.Open(filename_Mbands, GA_ReadOnly)
    data_Abands = gdal.Open(filename_Abands, GA_ReadOnly)
    data_Pbands = gdal.Open(filename_Pbands, GA_ReadOnly)
   
    print ('3 bands raster size for %s' %(image_id))
    print data_3bands.RasterXSize, 'x', data_3bands.RasterYSize, 'x' ,data_3bands.RasterCount
    print '\n'
    print ('M bands raster size for %s' %(image_id))
    print data_Mbands.RasterXSize, 'x' , data_Mbands.RasterYSize, 'x', data_Mbands.RasterCount
    print '\n'
    print ('A bands raster size for %s' %(image_id))
    print data_Abands.RasterXSize, 'x' , data_Abands.RasterYSize, 'x', data_Abands.RasterCount
    print '\n'
    print ('p bands raster size for %s' %(image_id))
    print data_Pbands.RasterXSize, 'x' , data_Pbands.RasterYSize, 'x', data_Pbands.RasterCount
    
    
    

In [53]:
print 'Show rastersize for all 25 training images\n'
for train_image_id in train_list:
    view_rastersize(train_image_id)
    print '\n'

Show rastersize for all 25 training images

3 bands raster size for 6040_2_2
3391 x 3349 x 3


M bands raster size for 6040_2_2
848 x 837 x 8


A bands raster size for 6040_2_2
136 x 134 x 8


p bands raster size for 6040_2_2
3391 x 3348 x 1


3 bands raster size for 6120_2_2
3403 x 3348 x 3


M bands raster size for 6120_2_2
851 x 837 x 8


A bands raster size for 6120_2_2
137 x 134 x 8


p bands raster size for 6120_2_2
3403 x 3348 x 1


3 bands raster size for 6120_2_0
3403 x 3348 x 3


M bands raster size for 6120_2_0
851 x 837 x 8


A bands raster size for 6120_2_0
137 x 134 x 8


p bands raster size for 6120_2_0
3403 x 3348 x 1


3 bands raster size for 6090_2_0
3393 x 3349 x 3


M bands raster size for 6090_2_0
848 x 837 x 8


A bands raster size for 6090_2_0
136 x 134 x 8


p bands raster size for 6090_2_0
3392 x 3348 x 1


3 bands raster size for 6040_1_3
3391 x 3349 x 3


M bands raster size for 6040_1_3
848 x 837 x 8


A bands raster size for 6040_1_3
136 x 134 x 8


p bands

In [55]:
view_rastersize(train_list[0])

3 bands raster size for 6040_2_2
3391 x 3349 x 3


M bands raster size for 6040_2_2
848 x 837 x 8


A bands raster size for 6040_2_2
136 x 134 x 8


p bands raster size for 6040_2_2
3391 x 3348 x 1


In [56]:
view_rastersize(train_list[1])

3 bands raster size for 6120_2_2
3403 x 3348 x 3


M bands raster size for 6120_2_2
851 x 837 x 8


A bands raster size for 6120_2_2
137 x 134 x 8


p bands raster size for 6120_2_2
3403 x 3348 x 1


In [57]:
view_rastersize(train_list[2])

3 bands raster size for 6120_2_0
3403 x 3348 x 3


M bands raster size for 6120_2_0
851 x 837 x 8


A bands raster size for 6120_2_0
137 x 134 x 8


p bands raster size for 6120_2_0
3403 x 3348 x 1


In [58]:
view_rastersize(train_list[3])

3 bands raster size for 6090_2_0
3393 x 3349 x 3


M bands raster size for 6090_2_0
848 x 837 x 8


A bands raster size for 6090_2_0
136 x 134 x 8


p bands raster size for 6090_2_0
3392 x 3348 x 1


In [59]:
view_rastersize(train_list[4])

3 bands raster size for 6040_1_3
3391 x 3349 x 3


M bands raster size for 6040_1_3
848 x 837 x 8


A bands raster size for 6040_1_3
136 x 134 x 8


p bands raster size for 6040_1_3
3391 x 3348 x 1


In [60]:
view_rastersize(train_list[5])

3 bands raster size for 6040_1_0
3391 x 3349 x 3


M bands raster size for 6040_1_0
848 x 837 x 8


A bands raster size for 6040_1_0
136 x 134 x 8


p bands raster size for 6040_1_0
3391 x 3348 x 1


In [61]:
view_rastersize(train_list[6])

3 bands raster size for 6100_1_3
3391 x 3349 x 3


M bands raster size for 6100_1_3
848 x 837 x 8


A bands raster size for 6100_1_3
136 x 134 x 8


p bands raster size for 6100_1_3
3391 x 3348 x 1


In [62]:
view_rastersize(train_list[7])

3 bands raster size for 6010_4_2
3396 x 3345 x 3


M bands raster size for 6010_4_2
849 x 837 x 8


A bands raster size for 6010_4_2
136 x 134 x 8


p bands raster size for 6010_4_2
3396 x 3348 x 1


In [63]:
view_rastersize(train_list[8])

3 bands raster size for 6110_4_0
3396 x 3348 x 3


M bands raster size for 6110_4_0
849 x 837 x 8


A bands raster size for 6110_4_0
136 x 134 x 8


p bands raster size for 6110_4_0
3396 x 3348 x 1


In [64]:
view_rastersize(train_list[9])

3 bands raster size for 6140_3_1
3396 x 3348 x 3


M bands raster size for 6140_3_1
849 x 837 x 8


A bands raster size for 6140_3_1
136 x 134 x 8


p bands raster size for 6140_3_1
3396 x 3348 x 1


In [65]:
view_rastersize(train_list[10])

3 bands raster size for 6110_1_2
3396 x 3348 x 3


M bands raster size for 6110_1_2
849 x 837 x 8


A bands raster size for 6110_1_2
136 x 134 x 8


p bands raster size for 6110_1_2
3396 x 3348 x 1


In [66]:
view_rastersize(train_list[11])

3 bands raster size for 6100_2_3
3391 x 3349 x 3


M bands raster size for 6100_2_3
848 x 837 x 8


A bands raster size for 6100_2_3
136 x 134 x 8


p bands raster size for 6100_2_3
3391 x 3348 x 1


In [67]:
view_rastersize(train_list[12])

3 bands raster size for 6150_2_3
3403 x 3348 x 3


M bands raster size for 6150_2_3
851 x 837 x 8


A bands raster size for 6150_2_3
137 x 134 x 8


p bands raster size for 6150_2_3
3403 x 3348 x 1


In [68]:
view_rastersize(train_list[13])

3 bands raster size for 6160_2_1
3393 x 3349 x 3


M bands raster size for 6160_2_1
848 x 837 x 8


A bands raster size for 6160_2_1
136 x 134 x 8


p bands raster size for 6160_2_1
3392 x 3348 x 1


In [69]:
view_rastersize(train_list[14])

3 bands raster size for 6140_1_2
3396 x 3348 x 3


M bands raster size for 6140_1_2
849 x 837 x 8


A bands raster size for 6140_1_2
136 x 134 x 8


p bands raster size for 6140_1_2
3396 x 3348 x 1


In [70]:
view_rastersize(train_list[15])

3 bands raster size for 6110_3_1
3396 x 3348 x 3


M bands raster size for 6110_3_1
849 x 837 x 8


A bands raster size for 6110_3_1
136 x 134 x 8


p bands raster size for 6110_3_1
3396 x 3348 x 1


In [71]:
view_rastersize(train_list[16])

3 bands raster size for 6010_4_4
3393 x 3345 x 3


M bands raster size for 6010_4_4
848 x 837 x 8


A bands raster size for 6010_4_4
135 x 134 x 8


p bands raster size for 6010_4_4
3392 x 3348 x 1


In [72]:
view_rastersize(train_list[17])

3 bands raster size for 6170_2_4
3389 x 3349 x 3


M bands raster size for 6170_2_4
848 x 837 x 8


A bands raster size for 6170_2_4
135 x 134 x 8


p bands raster size for 6170_2_4
3392 x 3348 x 1


In [73]:
view_rastersize(train_list[18])

3 bands raster size for 6170_4_1
3393 x 3345 x 3


M bands raster size for 6170_4_1
848 x 837 x 8


A bands raster size for 6170_4_1
136 x 134 x 8


p bands raster size for 6170_4_1
3392 x 3348 x 1


In [74]:
view_rastersize(train_list[19])

3 bands raster size for 6170_0_4
3389 x 3349 x 3


M bands raster size for 6170_0_4
848 x 837 x 8


A bands raster size for 6170_0_4
135 x 134 x 8


p bands raster size for 6170_0_4
3392 x 3348 x 1


In [75]:
view_rastersize(train_list[20])

3 bands raster size for 6060_2_3
3403 x 3348 x 3


M bands raster size for 6060_2_3
851 x 837 x 8


A bands raster size for 6060_2_3
136 x 134 x 8


p bands raster size for 6060_2_3
3404 x 3348 x 1


In [76]:
view_rastersize(train_list[21])

3 bands raster size for 6070_2_3
3338 x 3350 x 3


M bands raster size for 6070_2_3
835 x 838 x 8


A bands raster size for 6070_2_3
134 x 134 x 8


p bands raster size for 6070_2_3
3338 x 3349 x 1


In [77]:
view_rastersize(train_list[22])

3 bands raster size for 6010_1_2
3396 x 3349 x 3


M bands raster size for 6010_1_2
849 x 837 x 8


A bands raster size for 6010_1_2
136 x 134 x 8


p bands raster size for 6010_1_2
3396 x 3348 x 1


In [78]:
view_rastersize(train_list[23])

3 bands raster size for 6040_4_4
3387 x 3346 x 3


M bands raster size for 6040_4_4
846 x 837 x 8


A bands raster size for 6040_4_4
134 x 133 x 8


p bands raster size for 6040_4_4
3388 x 3348 x 1


In [79]:
view_rastersize(train_list[24])

3 bands raster size for 6100_2_2
3391 x 3349 x 3


M bands raster size for 6100_2_2
848 x 837 x 8


A bands raster size for 6100_2_2
136 x 134 x 8


p bands raster size for 6100_2_2
3391 x 3348 x 1


# Try unify the size of the image

In [93]:
def size_transfer (image_id):
    
    
    image_initial = tiff.imread('satellite_image/sixteen_band/{}.tif'.format(image_id + '_A')).transpose([1, 2, 0])
    print image_initial.shape
    initial_x1 = image_initial.shape[0]
    initial_x2 = image_initial.shape[1]
    print ("Initial size for %s is %d x %d" %(image_id,initial_x1,initial_x2))
    
    image_final = tiff.imread('satellite_image/sixteen_band/{}.tif'.format(image_id + '_M')).transpose([1, 2, 0])
    print image_final.shape
    final_x1 = image_final.shape[0]
    final_x2 = image_final.shape[1]
    print ("Final size for %s is %d x %d" %(image_id,final_x1,final_x2))
    
    
    
    
    

In [94]:
size_transfer (train_list[0])

(134, 136, 8)
Initial size for 6040_2_2 is 134 x 136
(837, 848, 8)
Final size for 6040_2_2 is 837 x 848


In [104]:
A = np.zeros((6,4))
print A
A1 = 6
A2 = 4

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [105]:
B = np.array([[1,3],[2,4],[5,7]])
B1 = 3
B2 =2

In [106]:
print B

[[1 3]
 [2 4]
 [5 7]]


In [109]:
print (A1/B1)

2.0
